In [6]:
import numpy as np
import sklearn
import underthesea
import json
import os
from tqdm.autonotebook import tqdm
from glob import glob
from pprint import pprint
import nltk
from underthesea import word_tokenize
from sklearn.decomposition import TruncatedSVD
import math
import sklearn


/Users/nguyenthanhtrung/opt/anaconda3/envs/recommend_env_2/lib/python3.6/site-packages/ipykernel_launcher.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


## Load data in json file

In dataset, item format is json with many fields and coresponding content 

In [2]:
json_fns = glob("./dataset/*.json")
items = []

for fn in json_fns:
    with open(fn, "r") as f:
        items += json.load(f)
print(len(items))
test_item = items[0]
pprint(test_item)

3459
{'Danh Mục': 'danh mục thời trang nữ phụ kiện may mặc',
 'Gửi từ': 'gửi từ quận hai bà trưng hà nội',
 'Kho hàng': 'kho hàng 33933',
 'Thương hiệu': 'thương hiệu no brand',
 'desc_2': 'vải voan lươi mêm loại 1 khổ vải rộng 17m chiều dài khách đặt số '
           'lượng bao nhiêu cắt khổ vải dài bấy nhiêu ví dụ khách ấn màu đỏ '
           'đặt mua số lượng 10 thì em sẽ cắt 1 tấm vái dài 10m rộng 17m  chất '
           'liệu mềm mịn độ phồng cao có nhiều màu sắc phong phú cho khách lựa '
           'chọn vì chỉ đăng được 20 lựa chọn màu khách muốn lấy màu nào không '
           'có trong lựa chọn thì ib em nhé xanh ngọc có ngọc đậm và ngọc nhạt '
           'xanh biển đậm xanh biển nhạt hồng phấn có đậm và nhạt vàng có vàng '
           'đậm tức vàng cúc và vàng nhạt voan lưới thích hợp dùng làm váy '
           'tutu cho trẻ em dùng may váy người lớn làm đồ handmade trang trí '
           'tiệc sinh nhật tiệc cưới liên hê 0369 359 588 lưu ý tổng đơn hàng '
           'ít nhất từ 2

### Tokenizer: word segmentation

In [3]:
def word_tokenizer_item(item):
    '''
    Word tokenize for all field in a item
    '''
    for key in item.keys():
        item[key] = word_tokenize(item[key], format='text')
    return item

In [4]:
def count_unigram(text):
    '''
    Count appearance number for each vocabulary
    '''
    counter = {}
    words = text.split()
    vocabs = set(words)
    for vocab in vocabs:
        if not vocab.isdigit():
            counter[vocab] = words.count(vocab)
    return counter

def combine_metadata(item):
    '''
    Combine all data field of item to 2 main field: product name and product description
    Output format: item = {
        name: 'product name',
        desc: 'product description'
    }
    '''
    newItem = {}
    newItem['name'] = item['title']
    newItem['desc'] = ""
    if 'Thương hiệu' in item.keys():
        if "no brand" not in item['Thương hiệu']:
            newItem['name'] += f" . {item['Thương hiệu']}"
    for key in item.keys():
        newItem["desc"] += f" . {item[key]}"
    return newItem
    
def count_word_in_dataset(items):
    '''
    count number of word in dataset
    '''
    nameCounter = {}
    descCounter = {}
    for item in items:
        for word in item['unigram_name'].keys():
            if word in nameCounter.keys():
                nameCounter[word] += 1
            else:
                nameCounter[word] = 1
        for word in item['unigram_desc'].keys():
            if word in descCounter.keys():
                descCounter[word] += 1
            else:
                descCounter[word] = 1
    return nameCounter, descCounter

In [7]:
newItems = []
for item in tqdm(items):
    newItem = word_tokenizer_item(item)
    newItem = combine_metadata(newItem)
    newItem['unigram_name'] = count_unigram(newItem['name'])
    newItem['unigram_desc'] = count_unigram(newItem['desc'])
    newItems.append(newItem)

In [8]:
def tfidf(doc_len, corpus_len, doc_counter, corpus_counter, k=2):
    vector_len = len(corpus_counter)
    tfidf_vector = np.zeros((vector_len,))
    for i, key in enumerate(corpus_counter.keys()):
        if key in doc_counter.keys():
            tf = (k+1)*doc_counter[key]/(k+doc_counter[key])
            idf = math.log((corpus_len+1)/(corpus_counter[key]))
            tfidf_vector[i] = tf*idf
    return tfidf_vector

### Word_counter in all corpus

In [9]:
nameCounter, descCounter = count_word_in_dataset(newItems)
with open("name_counter.json", "w+", encoding='utf8') as f:
    json.dump(nameCounter, f, sort_keys=True, indent=2, ensure_ascii=False)
    
with open("desc_counter.json", "w+", encoding='utf8') as f:
    json.dump(descCounter, f, sort_keys=True, indent=2, ensure_ascii=False)

### Tf-idf 

In [10]:
tfidf_vectors = []
corpus_len = len(newItems)
for item in tqdm(newItems):
    doc_len = len(item['desc'])
    tfidf_vectors.append(
        tfidf(doc_len, corpus_len, item['unigram_desc'], descCounter)
    )
print(len(tfidf_vectors))


3459


## SVD

In [13]:
svd = TruncatedSVD(n_components=256)
svd.fit(tfidf_vectors)
svd_tfidf_vector = svd.transform(tfidf_vectors)

In [14]:
query_vector = tfidf_vectors[0]
query_vector = np.reshape(query_vector, (1,-1))
# search
sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(query_vector, tfidf_vectors)
sim_maxtrix = np.reshape(sim_maxtrix, (-1,))

In [16]:

idx = (-sim_maxtrix).argsort()[:20]
for _id in idx:
    print(_id, sim_maxtrix[_id])
    print(newItems[_id]['name'].upper())
    print(newItems[_id]['desc'], "\n\n")


0 0.9999999999999998
VẢI VOAN LƯỚI MỀM LOẠI 1 15K 1M
 . danh_mục thời_trang nữ phụ_kiện may_mặc . gửi từ quận_hai bà trưng hà nội . kho hàng 33933 . thương_hiệu no brand . vải voan lươi mêm loại 1 khổ vải rộng 17m chiều dài khách đặt số_lượng bao_nhiêu cắt khổ vải dài bấy_nhiêu ví_dụ khách ấn màu đỏ đặt mua số_lượng 10 thì em sẽ cắt 1 tấm vái_dài 10m rộng 17m chất_liệu mềm mịn độ phồng cao có nhiều màu_sắc phong_phú cho khách lựa_chọn vì chỉ đăng được 20 lựa_chọn màu khách muốn lấy màu nào không có trong lựa_chọn thì ib em nhé xanh ngọc có ngọc đậm và ngọc nhạt xanh biển đậm xanh biển nhạt hồng phấn có đậm và nhạt vàng có vàng đậm tức vàng cúc và vàng nhạt voan lưới thích_hợp dùng làm váy tutu cho trẻ_em dùng may váy người_lớn làm đồ handmade trang_trí tiệc sinh_nhật tiệc cưới liên hê 0369 359 588 lưu_ý tổng đơn hàng ít_nhất từ 2m trở lên shop mới cắt vải voanluoi vaivoan luoitutu vaycongchua vảivoanlưới lướitutu giasi phukienmaymặc handmade phôngnền . 15000 . vải voan lưới mềm loại 1 